<a href="https://colab.research.google.com/github/krisivarga/deep_learning_HW_big/blob/main/LSTM_Text_Class_tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#download file from web

import requests, zipfile, io, os

zip_file_url = "http://groups.di.unipi.it/~gulli/newsspace200.xml.bz"

filename = zip_file_url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(zip_file_url)
    f.write(r.content)

In [34]:
#create data folder, decompress data

import bz2,shutil

dirName = 'data'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")
    
    
with bz2.BZ2File("newsspace200.xml.bz") as fr, open("./data/input.xml","wb") as fw:
    shutil.copyfileobj(fr,fw)

Directory  data  already exists


In [35]:
import xml.etree.ElementTree as ET

tree = ET.parse("./data/input.xml")
root = tree.getroot()

titles = []
categories = []
descriptions = []
sources = []


for title in tree.findall('title'):
    titles.append(title.text)
    
for category in tree.findall('category'):
        categories.append(category.text)

for description in tree.findall('description'):
    descriptions.append(description.text)

for source in tree.findall('source'):
    sources.append(source.text)


In [36]:
import numpy as np

In [37]:
titles = np.array(titles).T
categories = np.array(categories).T
descriptions = np.array(descriptions).T
sources = np.array(sources).T

In [38]:
data = np.dstack((sources,titles,descriptions,categories)).reshape(len(titles),4)

In [39]:
import pandas as pd

df = pd.DataFrame(data)
df.columns = ['source','title', 'desc', 'cat']

In [40]:
#set number of categories to be selected
num_selected = 5
selected_cats = df['cat'].value_counts()[:num_selected].index.tolist()
df_selected = df.loc[df['cat'].isin(selected_cats)]

In [41]:
df_selected = df_selected.dropna()

In [42]:
!pip install sentencepiece
!pip install transformers
from transformers import XLNetTokenizer
from transformers import BertTokenizer
import re

regex = re.compile('^[a-zA-Z]{3,}$')

xltokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

btokenizer = BertTokenizer.from_pretrained("bert-base-uncased")



In [43]:
def tr(bt,reg,title,sz):
  a = bt.tokenize(title + " " + sz)
  r = []
  for s in a:
    if reg.match(s) is not None:     
      r.append(s)      
  return r

df_selected['berttoken'] = df_selected.apply(lambda row: tr(btokenizer, regex, str(row['title']), str(row['desc'])), axis=1)

In [44]:
def bertenc(bt,title,sz):
  text_to_encode = title + " " + sz
  a = bt.encode_plus(
            text_to_encode, 
            max_length= 64, 
            add_special_tokens=True,
            return_token_type_ids=False, 
            padding="max_length",
            truncation = True,
            return_attention_mask=False
        )
  ids = a['input_ids']  
  return ids

df_selected['bertencodeplus'] = df_selected.apply(lambda row: bertenc(btokenizer, str(row['title']), str(row['desc'])), axis=1)


In [45]:
bertencidvoc_dict = {}
bertvoc_dict = {}
def dict_creater(arr):
  for a in arr:
    if a != 0:
      if a in bertencidvoc_dict:
        count = bertencidvoc_dict[a] + 1
        bertencidvoc_dict.update({a:count})
      else:
        bertencidvoc_dict.update({a:1})
  return

def dict_creater2(arr):
  for a in arr:
    if a != 0:
      if a in bertvoc_dict:
        count = bertvoc_dict[a] + 1
        bertvoc_dict.update({a:count})
      else:
        bertvoc_dict.update({a:1})
  return

df_selected.apply(lambda row: dict_creater(row['bertencodeplus']), axis=1)

df_selected.apply(lambda row: dict_creater2(row['berttoken']), axis=1)


0         None
1         None
2         None
3         None
4         None
          ... 
496829    None
496830    None
496831    None
496832    None
496833    None
Length: 324797, dtype: object

In [46]:
i = 0
category_dictionary = {}
cat_rev_dict= {}
for s in selected_cats:
  category_dictionary.update({i:s})
  cat_rev_dict.update({s:i})
  i = i + 1


df_selected['cat_enc'] = df_selected.apply(lambda row: cat_rev_dict[str(row['cat'])] , axis=1)


In [51]:
import tensorflow
from tensorflow import keras


In [49]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1dIsf_-kNjWo6vRtm_DPsbebZpZRhEzzd' -O 'best_LSTM_CONV1D_model.h5'

--2021-12-12 10:51:18--  https://docs.google.com/uc?export=download&id=1dIsf_-kNjWo6vRtm_DPsbebZpZRhEzzd
Resolving docs.google.com (docs.google.com)... 74.125.204.102, 74.125.204.139, 74.125.204.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.204.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-68-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6r46madb5o94vqgglcf244mf90vaae1s/1639306275000/16722052510491764536/*/1dIsf_-kNjWo6vRtm_DPsbebZpZRhEzzd?e=download [following]
--2021-12-12 10:51:19--  https://doc-0k-68-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/6r46madb5o94vqgglcf244mf90vaae1s/1639306275000/16722052510491764536/*/1dIsf_-kNjWo6vRtm_DPsbebZpZRhEzzd?e=download
Resolving doc-0k-68-docs.googleusercontent.com (doc-0k-68-docs.googleusercontent.com)... 142.250.157.132, 2404:6800:4008:c13::84
Connecting to doc-0k-68-docs.googleusercontent.com (doc-0k-68-

In [50]:
best_model = keras.models.load_model("best_LSTM_CONV1D_model.h5")

In [53]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 64, 128)           2602496   
                                                                 
 conv1d (Conv1D)             (None, 64, 10)            20490     
                                                                 
 dropout (Dropout)           (None, 64, 10)            0         
                                                                 
 lstm (LSTM)                 (None, 112)               55104     
                                                                 
 dropout_1 (Dropout)         (None, 112)               0         
                                                                 
 dense (Dense)               (None, 64)                7232      
                                                                 
 dense_1 (Dense)             (None, 5)                 3

In [54]:
test_articles = df_selected.sample(10)

In [55]:
test_articles

,source,title,desc,cat,berttoken,bertencodeplus,cat_enc
427714,Scotland on Sunday,Aid Group Threatens Afghanistan Pullout,An aid group in Afghanistan said it was consid...,World,"[aid, group, threatens, afghanistan, pull, aid...","[101, 4681, 2177, 17016, 7041, 4139, 5833, 201...",0
353401,Boston Globe,Advancing forces meet sporadic resistance,"US and Iraqi forces, facing sporadic but stiff...",Entertainment,"[advancing, forces, meet, sporadic, resistance...","[101, 10787, 2749, 3113, 24590, 5012, 2149, 19...",1
130786,Tucson Citizen,Han wins LPGA event in playoff over Kane,Han made a 4-foot birdie putt on the first pla...,Sports,"[han, wins, event, playoff, over, kane, han, m...","[101, 7658, 5222, 6948, 3654, 2724, 1999, 7808...",2
416112,Telegraph.co.uk,Hartson #39;s goal gives Celtic hope in Europe,They may not have exorcised their demons but C...,Sports,"[hart, goal, gives, celtic, hope, europe, they...","[101, 7530, 3385, 1001, 4464, 1025, 1055, 3125...",2
445740,New York Times Movies,Horror Minus Perspective in a Tale of the Holo...,"This slow, earnest picture is a minor example ...",Entertainment,"[horror, minus, perspective, tale, the, holoca...","[101, 5469, 15718, 7339, 1999, 1037, 6925, 199...",1
452919,MLB.com,Padres make offer to Wells,com. The Padres offered salary arbitration to ...,Sports,"[padres, make, offer, wells, com, the, padres,...","[101, 21577, 2191, 3749, 2000, 7051, 4012, 101...",2
402279,Soccer Central,Reid striker not enough as Rovers stay rooted,Ireland International Steven Reid scored for B...,Sports,"[reid, striker, not, enough, rovers, stay, roo...","[101, 9027, 11854, 2025, 2438, 2004, 9819, 299...",2
268608,Sports Network,(Sports Network) - The Boston Red Sox are atte...,from an 0-3 deficit to win a best-of-seven ser...,Sports,"[sports, network, the, boston, red, sox, are, ...","[101, 1006, 2998, 2897, 1007, 1011, 1996, 3731...",2
320498,CBS Sportsline,Johansson edges Agassi in tiebreak to win Stoc...,"STOCKHOLM, Sweden -- Thomas Johansson rallied ...",Sports,"[johansson, edges, tie, win, stockholm, title,...","[101, 26447, 7926, 12943, 12054, 2072, 1999, 5...",2
478265,WNEP-TV,Obesity Drug Trials Get Good Results,Those patients also showed healthier cholester...,Top Stories,"[obesity, drug, trials, get, good, results, th...","[101, 24552, 4319, 7012, 2131, 2204, 3463, 221...",4


In [56]:
test_articles_x = np.array(test_articles["bertencodeplus"].values.tolist())

In [57]:
test_articles_y = test_articles['cat_enc'].to_numpy()

In [58]:
test_articles_preds = best_model.predict(test_articles_x)

In [59]:
classes_x=np.argmax(test_articles_preds,axis=1)

In [ ]:
classes_x

array([1, 2, 0, 2, 3, 3, 0, 0, 0, 1])

In [60]:
category_dictionary

{0: 'World', 1: 'Entertainment', 2: 'Sports', 3: 'Business', 4: 'Top Stories'}

In [61]:
examples = test_articles[["title", "desc", "cat"]]

In [62]:
examples["preds"] = classes_x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
examples["preds"] = examples["preds"].apply(lambda x: category_dictionary.get(x))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
examples

,title,desc,cat,preds
427714,Aid Group Threatens Afghanistan Pullout,An aid group in Afghanistan said it was consid...,World,World
353401,Advancing forces meet sporadic resistance,"US and Iraqi forces, facing sporadic but stiff...",Entertainment,World
130786,Han wins LPGA event in playoff over Kane,Han made a 4-foot birdie putt on the first pla...,Sports,Sports
416112,Hartson #39;s goal gives Celtic hope in Europe,They may not have exorcised their demons but C...,Sports,Sports
445740,Horror Minus Perspective in a Tale of the Holo...,"This slow, earnest picture is a minor example ...",Entertainment,Entertainment
452919,Padres make offer to Wells,com. The Padres offered salary arbitration to ...,Sports,Sports
402279,Reid striker not enough as Rovers stay rooted,Ireland International Steven Reid scored for B...,Sports,Sports
268608,(Sports Network) - The Boston Red Sox are atte...,from an 0-3 deficit to win a best-of-seven ser...,Sports,Sports
320498,Johansson edges Agassi in tiebreak to win Stoc...,"STOCKHOLM, Sweden -- Thomas Johansson rallied ...",Sports,Sports
478265,Obesity Drug Trials Get Good Results,Those patients also showed healthier cholester...,Top Stories,Entertainment
